In [1]:
import sys
import os
import math
import pandas as pd
import numpy as np
import json
from math import sqrt
from collections import defaultdict

#import recmetrics

# Python visualisation library
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Another much simpler viz library
import seaborn as sns

# Modelling Libaries
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
import sklearn.metrics
from surprise import Reader, Dataset, SVD, evaluate
from collections import defaultdict
from surprise.model_selection import KFold, train_test_split

# Adding root of git repo to path so we can import pyfireanalytics
sys.path.append('..')

# Display formatting options
pd.set_option('display.max_columns', 1000)

In [2]:
data = pd.read_csv('~/desktop/movielens/UDATA.csv', encoding="latin-1")
item = pd.read_csv('~/desktop/movielens/UITEM.csv', encoding="latin-1")
genre = pd.read_csv('~/desktop/movielens/UGENRE.csv', encoding="latin-1")
user = pd.read_csv('~/desktop/movielens/UUSER.csv', encoding="latin-1")
test = pd.read_csv('~/desktop/movielens/U1TEST.csv', encoding="latin-1")
train = pd.read_csv('~/desktop/movielens/U1BASE.csv', encoding="latin-1")

In [3]:
# rename itemid to movieid to join data and item together
data.rename(columns={'itemid':'movieid'}, inplace=True)
test.rename(columns={'itemid':'movieid'}, inplace=True)
train.rename(columns={'itemid':'movieid'}, inplace=True)
#join
df = pd.merge(data, item, on='movieid', how='outer')
df = pd.merge(df, user, on='userid', how='outer')

user2 = pd.merge(user, data, on='userid', how='inner')
user2 = user2.drop('timestamp', 1)
user2 = user2.drop_duplicates()
#change unix timestamp to readable date
df['reviewtimestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Split reviewtimestamp into year month day
df['reviewyear'] = df['reviewtimestamp'].dt.year
df['reviewmonth'] = df['reviewtimestamp'].dt.month
df['reviewday'] = df['reviewtimestamp'].dt.day


In [5]:
item.head(10)

,movieid,movietitle,releasedate,imdburl,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,1,Toy Story (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0
1,2,GoldenEye (1995),1-Jan-95,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0
2,3,Four Rooms (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.0,0.0
3,4,Get Shorty (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
4,5,Copycat (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0.0,0.0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,1-Jan-95,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
6,7,Twelve Monkeys (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0.0,0.0
7,8,Babe (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
8,9,Dead Man Walking (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,0.0
9,10,Richard III (1995),22-Jan-96,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0


In [6]:
genres = ['unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama',
         'fantasy', 'filmnoir', 'horror', 'musical', 'mystery', 'romance', 'scifi', 'thriller', 'war', 'western']

def getunknown(item):
    if (item['unknown'] == 1 ):
         return 'unknown'
    else:
         return 0

def getaction(item):
    if (item['action'] == 1 ):
         return 'action'
    else:
         return 0

def getadventure(item):
    if (item['adventure'] == 1 ):
         return 'adventure'
    else:
         return 0

def getanimation(item):
    if (item['animation'] == 1 ):
         return 'animation'
    else:
         return 0
        
def getchildrens(item):
    if (item['childrens'] == 1 ):
         return 'childrens'
    else:
         return 0

def getcomedy(item):
    if (item['comedy'] == 1 ):
         return 'comedy'
    else:
         return 0

def getcrime(item):
    if (item['crime'] == 1 ):
         return 'crime'
    else:
         return 0

def getdocumentary(item):
    if (item['documentary'] == 1 ):
         return 'documentary'
    else:
         return 0

def getdrama(item):
    if (item['drama'] == 1 ):
         return 'drama'
    else:
         return 0

def getfantasy(item):
    if (item['fantasy'] == 1 ):
         return 'fantasy'
    else:
         return 0

def getfilmnoir(item):
    if (item['filmnoir'] == 1 ):
         return 'filmnoir'
    else:
         return 0

def gethorror(item):
    if (item['horror'] == 1 ):
         return 'horror'
    else:
         return 0
        
def getmusical(item):
    if (item['musical'] == 1 ):
         return 'musical'
    else:
         return 0

def getmystery(item):
    if (item['mystery'] == 1 ):
         return 'mystery'
    else:
         return 0

def getromance(item):
    if (item['romance'] == 1 ):
         return 'romance'
    else:
         return 0

def getscifi(item):
    if (item['scifi'] == 1 ):
         return 'scifi'
    else:
         return 0
        
def getthriller(item):
    if (item['thriller'] == 1 ):
         return 'thriller'
    else:
         return 0

def getwar(item):
    if (item['war'] == 1 ):
         return 'war'
    else:
         return 0
        
def getwestern(item):
    if (item['western'] == 1 ):
         return 'western'
    else:
         return 0
        
item['unknown'] = item.apply(getunknown, axis=1).astype(str)
item['action'] = item.apply(getaction, axis=1).astype(str)
item['adventure'] = item.apply(getadventure, axis=1).astype(str)
item['animation'] = item.apply(getanimation, axis=1).astype(str)
item['childrens'] = item.apply(getchildrens, axis=1).astype(str)
item['comedy'] = item.apply(getcomedy, axis=1).astype(str)
item['crime'] = item.apply(getcrime, axis=1).astype(str)
item['documentary'] = item.apply(getdocumentary, axis=1).astype(str)
item['drama'] = item.apply(getdrama, axis=1).astype(str)
item['fantasy'] = item.apply(getfantasy, axis=1).astype(str)
item['filmnoir'] = item.apply(getfilmnoir, axis=1).astype(str)
item['horror'] = item.apply(gethorror, axis=1).astype(str)
item['musical'] = item.apply(getmusical, axis=1).astype(str)
item['mystery'] = item.apply(getmystery, axis=1).astype(str)
item['romance'] = item.apply(getromance, axis=1).astype(str)
item['scifi'] = item.apply(getscifi, axis=1).astype(str)
item['thriller'] = item.apply(getthriller, axis=1).astype(str)
item['war'] = item.apply(getwar, axis=1).astype(str)
item['western'] = item.apply(getwestern, axis=1).astype(str)



In [7]:
item['genres'] = item['unknown'] + ', '+ item['action'] + ', '+ item['adventure']  + ', '+ item['animation'] + ', ' + item['childrens'] + ', '+ item['comedy']  + ', '+ item['crime']  + ', '+ item['documentary'] + ', ' + item['drama'] + ', '+ item['fantasy']  + ', '+ item['filmnoir']  + ', '+ item['horror'] + ', ' + item['musical'] + ', '+ item['mystery']  + ', '+ item['romance']  + ', '+ item['scifi'] + ', ' + item['thriller']  + ', '+ item['war']  + ', '+ item['western']


In [8]:
#join
df = pd.merge(data, item, on='movieid', how='outer')
df = pd.merge(df, user, on='userid', how='outer')

In [15]:
item.head(10)

,movieid,movietitle,releasedate,imdburl,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western,genres
0,1,Toy Story (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,animation,childrens,comedy,0,0,0,0,0,0,0,0,0,0,0,0,0,"0, 0, 0, animation, childrens, comedy, 0, 0, 0..."
1,2,GoldenEye (1995),1-Jan-95,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,action,adventure,0,0,0,0,0,0,0,0,0,0,0,0,0,thriller,0,0,"0, action, adventure, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,Four Rooms (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,thriller,0,0,"0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3,4,Get Shorty (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,action,0,0,0,comedy,0,0,drama,0,0,0,0,0,0,0,0,0,0,"0, action, 0, 0, 0, comedy, 0, 0, drama, 0, 0,..."
4,5,Copycat (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,crime,0,drama,0,0,0,0,0,0,0,thriller,0,0,"0, 0, 0, 0, 0, 0, crime, 0, drama, 0, 0, 0, 0,..."
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,1-Jan-95,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,0,0,0,drama,0,0,0,0,0,0,0,0,0,0,"0, 0, 0, 0, 0, 0, 0, 0, drama, 0, 0, 0, 0, 0, ..."
6,7,Twelve Monkeys (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,0,0,0,drama,0,0,0,0,0,0,scifi,0,0,0,"0, 0, 0, 0, 0, 0, 0, 0, drama, 0, 0, 0, 0, 0, ..."
7,8,Babe (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,childrens,comedy,0,0,drama,0,0,0,0,0,0,0,0,0,0,"0, 0, 0, 0, childrens, comedy, 0, 0, drama, 0,..."
8,9,Dead Man Walking (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,0,0,0,drama,0,0,0,0,0,0,0,0,0,0,"0, 0, 0, 0, 0, 0, 0, 0, drama, 0, 0, 0, 0, 0, ..."
9,10,Richard III (1995),22-Jan-96,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,0,0,0,drama,0,0,0,0,0,0,0,0,war,0,"0, 0, 0, 0, 0, 0, 0, 0, drama, 0, 0, 0, 0, 0, ..."


# Model

In [16]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#Tfidf and Cosine similarity calc
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(item['genres'])

cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

results = {}

for idx, row in item.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], item['movieid'][i]) for i in similar_indices]
    results[row['movieid']] = similar_items[1:]


In [21]:
# Output movie name from movieid
def items(id):
    return item.loc[item['movieid'] == id]['movietitle'].tolist()[0]

# Read results generating in a list
def recommend(movieid, n):
    recs = results[movieid][:n]
    for rec in recs:
        print("Recommended: " + items(rec[1]) + " (score:" + str(rec[0]) + ")")


In [22]:
#input movieid and number of recommendations for recommendations
recommend(movieid=1, n=10)

Recommended: Aladdin and the King of Thieves (1996) (score:1.0)
Recommended: Goofy Movie, A (1995) (score:0.7746935703622502)
Recommended: Aladdin (1992) (score:0.7586574717504039)
Recommended: Gumby: The Movie (1995) (score:0.6789157439013491)
Recommended: Pinocchio (1940) (score:0.6789157439013491)
Recommended: Oliver & Company (1988) (score:0.6789157439013491)
Recommended: Balto (1995) (score:0.6789157439013491)
Recommended: Sword in the Stone, The (1963) (score:0.6789157439013491)
Recommended: Swan Princess, The (1994) (score:0.6789157439013491)
Recommended: Aristocats, The (1970) (score:0.6789157439013491)


In [23]:
item[item['movieid'] == 1]

,movieid,movietitle,releasedate,imdburl,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western,genres
0,1,Toy Story (1995),1-Jan-95,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,animation,childrens,comedy,0,0,0,0,0,0,0,0,0,0,0,0,0,"0, 0, 0, animation, childrens, comedy, 0, 0, 0..."


# VIZ

In [128]:
occupation = ['administrator', 'artist', 'doctor', 'educator', 'engineer', 'entertainment', 'executive', 'healthcare',
              'homemaker', 'lawyer', 'librarian', 'marketing', 'none', 'other', 'programmer', 'retired', 'salesman',
              'scientist', 'student', 'technician', 'writer']

for i in occupation:
    x = df[df['occupation'] == i]['rating'].mean()
    print(x)

3.6356464768017114
3.653379549393414
3.688888888888889
3.6706206312221985
3.541406727828746
3.4410501193317424
3.3491037320011756
2.896219686162625
3.301003344481605
3.7353159851301116
3.560781338896264
3.4856410256410255
3.779134295227525
3.5523773797242804
3.5682604794257147
3.4667495338719703
3.582943925233645
3.611273080660836
3.5151432345038027
3.5322304620650313
3.3757225433526012


In [129]:
genres = ['unknown', 'action', 'adventure', 'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama',
         'fantasy', 'filmnoir', 'horror', 'musical', 'mystery', 'romance', 'scifi', 'thriller', 'war', 'western']

for i in genres:
    x = df[df[i] == i]['rating'].mean()
    print(x)

2.3333333333333335
3.4802829561886894
3.503526503308369
3.5766990291262135
3.3532442216652742
3.3940734781442745
3.6322780881440098
3.6657894736842107
3.6875141002180833
3.2152366863905324
3.9215233698788228
3.2903893172841827
3.521396851029471
3.63813155386082
3.621704948358255
3.5607227022780834
3.5090069495245064
3.815811874866993
3.6132686084142396
